In [1]:
# Load the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import cv2
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import keras
from keras import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from matplotlib.image import imread

In [2]:
train_dir = "/kaggle/input/food10/train"
test_dir  = "/kaggle/input/food10/test"

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

target_size = (150,150)
batch_size = 16

# Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')

Found 6273 images belonging to 10 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=False,    
    class_mode='categorical')

Found 1798 images belonging to 10 classes.


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding = 'same', input_shape = (150, 150, 3)))
model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding= 'same', input_shape = (150, 150, 3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding = 'same'))
model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding = 'same'))
model.add(MaxPooling2D((3, 3)))
model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding = 'same'))
model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding = 'same'))
model.add(MaxPooling2D((3, 3)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation = 'softmax'))

2022-05-19 05:35:42.470363: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-19 05:35:42.571245: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-19 05:35:42.572095: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-19 05:35:42.573529: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
from tensorflow.keras.optimizers import SGD

opt = SGD(lr = 0.01, momentum = 0.9)
model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [7]:
from tensorflow.keras.optimizers import SGD

opt = SGD(lr = 0.01, momentum = 0.9)
model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(train_generator, batch_size = 128, epochs = 10, validation_data = test_generator, verbose = 1)

2022-05-19 05:35:45.404168: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-05-19 05:35:47.356259: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


393/393 [==============================] - 207s 508ms/step - loss: 2.2621 - accuracy: 0.1543 - val_loss: 2.1955 - val_accuracy: 0.1813
Epoch 2/10
393/393 [==============================] - 172s 437ms/step - loss: 2.1986 - accuracy: 0.1856 - val_loss: 2.2293 - val_accuracy: 0.1674
Epoch 3/10
393/393 [==============================] - 172s 437ms/step - loss: 2.1650 - accuracy: 0.2099 - val_loss: 2.1119 - val_accuracy: 0.2113
Epoch 4/10
393/393 [==============================] - 174s 443ms/step - loss: 2.1255 - accuracy: 0.2251 - val_loss: 2.0337 - val_accuracy: 0.2653
Epoch 5/10
393/393 [==============================] - 173s 440ms/step - loss: 2.0235 - accuracy: 0.2756 - val_loss: 2.1856 - val_accuracy: 0.2408
Epoch 6/10
393/393 [==============================] - 172s 436ms/step - loss: 1.9721 - accuracy: 0.2971 - val_loss: 2.0090 - val_accuracy: 0.2970
Epoch 7/10
393/393 [==============================] - 172s 437ms/step - loss: 1.9203 - accuracy: 0.3176 - val_loss: 1.7918 - val_accura

In [9]:
model.save('best_model.h5')

In [10]:
model.evaluate(test_generator)

113/113 [==============================] - 34s 303ms/step - loss: 1.7900 - accuracy: 0.3548


[1.7899606227874756, 0.35483869910240173]

In [11]:
classes = list(train_generator.class_indices.keys())
print(classes)
print(len(classes))

['Banh beo', 'Banh chung', 'Banh khot', 'Banh mi', 'Banh xeo', 'Bun dau mam tom', 'Canh chua', 'Com tam', 'Hu tieu', 'Pho']
10


In [12]:
import cv2
import numpy as np 

def predict(path):
  img = cv2.imread(path)
  img = img[:, :, ::-1]
  img = cv2.resize(img,(im_size, im_size))

  plt.imshow(img)

  x = np.reshape(img, (1, img_size, im_size, 3))
  p = model.predict(x / 255)
  # print(p)
  arg = np.argmax(p, axis=-1)[0]
  print('class:',classes[arg])
  print('prob:', p[0][arg])

In [13]:
path = 'kaggle/input/food10/test/Banh mi/1020.jpg'
predict(path)

TypeError: 'NoneType' object is not subscriptable